In [1]:
from ckks_package.ckks import *

In [ ]:
N = 2**15  # Ring degree
n = 2**2  # Number of slots
L = 20  # Maximal level
q0 = 2**40  # Smallest modulus
p = 2**33  # Scaling factor outside of bootstrapping

# The grouping parameter for bootstrapping is s. It lies between 1 and
# log(n, 2). The smaller s, the faster the bootstrapping, but the more levels
# are consumed.
s = log(n, 2)

CKKS.config(N, n, L, q0, p)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, s)

The CKKS configuration is done!

The key generation is done!
Estimated security: 2^(80.0).

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [3]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([-1.+0.j,  2.-2.j, -0.-3.j, -2.-1.j]),
 array([-3.-1.j,  3.+2.j,  2.+0.j,  2.+3.j])]

In [4]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[- 2147483648 - 4789838791X^4096 - 1518500250X^8192 + 13888113322X^12288 - 12884901888X^16384 - 340403362X^20480 + 1518500250X^24576 + 5752644894X^28672 mod(2^700),
 8589934592 - 13547709959X^4096 - 18222003000X^8192 - 3287225653X^12288 + 8589934592X^16384 + 962806103X^20480 + 7936064755X^28672 mod(2^700)]

In [5]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20),
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)]

In [6]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[-1.00000000e+00+1.18055343e-10j  2.00000000e+00-2.00000000e+00j
  2.06893336e-10-3.00000000e+00j -2.00000000e+00-9.99999999e-01j]
[-3.-1.00000000e+00j  3.+2.00000000e+00j  2.+3.77559428e-10j
  2.+3.00000000e+00j]


In [7]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^19 (level 19 out of 20)


In [8]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[-4.00000000e+00-1.00000000e+00j  5.00000000e+00-1.97827532e-09j
  2.00000000e+00-3.00000000e+00j  9.73898739e-10+2.00000000e+00j]
[ 3.00000000e+00+1.j          1.00000000e+01-2.j
  1.59706226e-09-6.j         -1.00000000e+00-7.99999999j]


In [9]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(s)
print(ct_boot)

pt = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^0 (level 0 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^2 (level 2 out of 20)
[-1.00054178e+00-2.58022709e-05j  1.99844423e+00-1.99800037e+00j
  4.00266877e-04-2.99684732e+00j -1.99826880e+00-9.99197736e-01j]
